# Stáhnutí meteodat pro stanici Praha-Ruzyně

(případně pro jiné město)

In [3]:
from datetime import datetime

import pandas as pd
from meteostat import Hourly, Stations

In [1]:
lat, lon = 50.0755, 14.4378  # Praha

In [ ]:
# Najděme nejbližší stanice
s = Stations()
s = s.nearby(
    lat=lat,
    lon=lon,
    radius=100000  # v m?
)
stations = s.fetch(10)
stations   # pandas DataFrame

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
11520,Praha-Libus,CZ,PR,11520,<NA>,50.0167,14.4500,304.0,Europe/Prague,NaT,NaT,1973-01-01,2025-08-24,1972-01-01,2021-01-01,6596.042516
11567,Praha Kbely,CZ,PR,11567,<NA>,50.1167,14.5333,286.0,Europe/Prague,2022-04-19,2022-04-23,2022-04-20,2022-04-22,NaT,NaT,8209.353634
11518,Praha / Ruzyne,CZ,ST,11518,LKPR,50.1000,14.2500,365.0,Europe/Prague,1929-08-02,2025-09-26,1940-01-01,2025-08-24,1940-01-01,2022-01-01,13672.589034
LKVO0,Vodochody,CZ,ST,<NA>,LKVO,50.2166,14.3958,280.0,Europe/Prague,2020-01-14,2025-09-23,NaT,NaT,NaT,NaT,15972.493875
11509,Doksany,CZ,US,11509,<NA>,50.4667,14.1667,158.0,Europe/Prague,NaT,NaT,NaT,NaT,NaT,NaT,47575.455792
11464,Milesovka,CZ,UK,11464,<NA>,50.5500,13.9333,836.0,Europe/Prague,NaT,NaT,1906-05-01,2025-08-24,1906-01-01,2021-01-01,63774.091397
11624,Caslav,CZ,ST,11624,LKCV,49.9333,15.3833,242.0,Europe/Prague,1997-01-01,2025-09-26,1997-03-09,2022-04-25,2005-01-01,2022-01-01,69397.954422
11628,Kramolin-Kosetice,CZ,VY,11628,<NA>,49.5833,15.0833,534.0,Europe/Prague,NaT,NaT,NaT,NaT,NaT,NaT,71687.081996
11502,Usti Nad Labem,CZ,US,11502,<NA>,50.6833,14.0333,377.0,Europe/Prague,NaT,NaT,NaT,NaT,NaT,NaT,73418.596224


In [5]:
# Použijeme třetí stanici v seznamu (Praha-Ruzyně)
station_id = stations.index[2]
station_id

'11518'

In [6]:
# Získáme hodinová data
station = stations.loc[station_id]
source = Hourly(station_id, start=station["hourly_start"], end=datetime(2024, 12, 31, 23, 59, 59))
source

In [8]:
df = source.fetch().reset_index()
df

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,1931-01-03 12:00:00,0.0,<NA>,<NA>,<NA>,<NA>,0.0,0.0,<NA>,1004.3,<NA>,<NA>
1,1931-01-03 18:00:00,1.1,<NA>,<NA>,<NA>,<NA>,0.0,0.0,<NA>,1002.8,<NA>,<NA>
2,1931-01-04 06:00:00,6.1,<NA>,<NA>,<NA>,<NA>,200.0,16.6,<NA>,1000.7,<NA>,<NA>
3,1931-01-04 12:00:00,11.1,<NA>,<NA>,<NA>,<NA>,160.0,3.6,<NA>,999.0,<NA>,<NA>
4,1931-01-04 18:00:00,6.1,<NA>,<NA>,<NA>,<NA>,270.0,33.5,<NA>,1001.8,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...
465779,2024-12-31 19:00:00,-5.7,-6.6,93.0,0.0,<NA>,140.0,3.6,20.4,1029.2,0.0,6.0
465780,2024-12-31 20:00:00,-4.3,-5.3,93.0,0.0,<NA>,180.0,7.2,22.2,1028.6,0.0,1.0
465781,2024-12-31 21:00:00,-2.5,-3.6,92.0,0.0,<NA>,180.0,10.8,22.2,1027.7,0.0,2.0
465782,2024-12-31 22:00:00,-2.8,-4.7,87.0,0.0,<NA>,190.0,10.8,22.2,1027.1,0.0,3.0


In [9]:
# Uložíme data do Parquet formátu
df.to_parquet(f"praha-meteostat.parquet")